In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#%matplotlib notebook
%matplotlib inline

In [3]:
import os
cwd = os.getcwd()

NOTEBOOK_DIR = os.path.dirname(cwd)
ROOT = os.path.dirname(os.path.dirname(os.path.dirname(NOTEBOOK_DIR)))

FIGURES_DIR = os.path.join(ROOT, 'figures/abc_parameterizations/training')

In [4]:
import sys
sys.path.append(ROOT)

In [20]:
import logging
from copy import deepcopy
import torch
import pickle
import pandas as pd
import matplotlib.pylab as pylab

from pytorch.job_runners.abc_parameterizations.abc_runner import ABCRunner
from pytorch.models.abc_params.fully_connected.ipllr import FcIPLLR
from utils.tools import read_yaml
from pytorch.configs.model import ModelConfig
from torch.utils.data import Dataset, Subset, DataLoader
from utils.plot.abc_parameterizations.results import *

In [8]:
CONFIG_PATH = os.path.join(ROOT, 'pytorch/configs/abc_parameterizations')
EXPERIMENTS_DIR = 'experiments'
MODEL_NAME = 'fc_ipllr'
CONFIG_FILE = 'fc_ipllr.yaml'

N_TRIALS = 5
L = 6  # n_layers - 1
WIDTH = 1024
N_WARMUP_STEPS = 1

In [9]:
activation="relu"
n_steps=600 
base_lr=0.01
batch_size=512
dataset="mnist"
download=False

## Setup and data

In [10]:
model_name = '{}_{}'.format(MODEL_NAME, dataset)
config_path = os.path.join(CONFIG_PATH, '{}.yaml'.format(model_name))
config_dict = read_yaml(config_path)

# define corresponding directory in experiments folder
base_experiment_path = os.path.join(ROOT, EXPERIMENTS_DIR, model_name)  # base experiment folder

# Load data & define models
logger = logging.getLogger()
logger.info('Loading data ...')
if dataset == 'mnist':
    from utils.dataset.mnist import load_data
elif dataset == 'cifar10':
    from utils.dataset.cifar10 import load_data
elif dataset == 'cifar100':
    # TODO : add cifar100 to utils.dataset
    pass
else:
    error = ValueError("dataset must be one of ['mnist', 'cifar10', 'cifar100'] but was {}".format(dataset))
    logger.error(error)
    raise error

# prepare data
training_dataset, test_dataset = load_data(download=download, flatten=True)
val_dataset = deepcopy(training_dataset)  # copy train_data into validation_data

In [21]:
train_data_loader = DataLoader(training_dataset, shuffle=True, batch_size=batch_size)
val_data_loader = DataLoader(val_dataset, shuffle=False, batch_size=batch_size)
test_data_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [13]:
config_dict['architecture']['width'] = WIDTH
config_dict['architecture']['n_layers'] = L + 1
config_dict['optimizer']['params']['lr'] = base_lr
config_dict['activation']['name'] = activation
config_dict['training']['n_steps'] = n_steps
config_dict['training']['batch_size'] = batch_size
config_dict['scheduler'] = {'name': 'warmup_switch',
                            'params': {'n_warmup_steps': N_WARMUP_STEPS,
                                       'calibrate_base_lr': True,
                                       'default_calibration': False}}

## Define config and model

In [17]:
config = ModelConfig(config_dict=config_dict)  # define the config as a class to pass to the model

In [22]:
lr_calibration_batches = list(train_data_loader)[:2]

In [23]:
model = FcIPLLR(config, n_warmup_steps=N_WARMUP_STEPS, lr_calibration_batches=lr_calibration_batches)

In [24]:
model.scheduler.initial_base_lrs

[0.01,
 15.408360481262207,
 36.97975540161133,
 38.14841079711914,
 40.09565353393555,
 44.029083251953125,
 9.410510063171387]

In [31]:
model.scheduler.current_lrs

[34359738368.0,
 1099511627776.0,
 1099511627776.0,
 1099511627776.0,
 1099511627776.0,
 1099511627776.0,
 34359738368.0]

In [33]:
model.scheduler.warm_lrs

[1024, 1048576, 1048576, 1048576, 1048576, 1048576, 1024]

In [26]:
[pg['lr'] for pg in model.optimizer.param_groups]

[343597383.68,
 16941671514112.0,
 40659671056384.0,
 41944621252608.0,
 44085637283840.0,
 48410488995840.0,
 323342663680.0]

In [27]:
[pg['lr'] for pg in model.scheduler.optimizer.param_groups]

[343597383.68,
 16941671514112.0,
 40659671056384.0,
 41944621252608.0,
 44085637283840.0,
 48410488995840.0,
 323342663680.0]

In [28]:
1024 **2

1048576

In [32]:
1099511627776.0 * 15.408360481262207

16941671514112.0

In [30]:
16941671514112.0

16941671514112.0